# AI Tools for Actuaries
## Chapter 3: Poisson GLM in Python
### Author: Mario Wuthrich
### Version March 2025

In [8]:
# Import required libraries
import pandas as pd
import numpy as np
import pyreadr
from statsmodels.formula.api import glm
import statsmodels.api as sm
import time

### Define Poisson Deviance Loss Function

In [9]:
def poisson_deviance(pred, obs):
    return 200 * (np.sum(pred) - np.sum(obs) + np.sum(np.log((obs/pred)**obs))) / len(pred)

### Load Data

In [10]:
# Load the data
dat = pyreadr.read_r('../../Data/freMTPL2freqClean.rda')  # load the data
dat = dat[None] if None in dat else dat[list(dat.keys())[0]]

# Display data structure
print(dat.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678007 entries, 0 to 678006
Data columns (total 14 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   IDpol       678007 non-null  float64 
 1   Exposure    678007 non-null  float64 
 2   Area        678007 non-null  category
 3   VehPower    678007 non-null  int32   
 4   VehAge      678007 non-null  int32   
 5   DrivAge     678007 non-null  int32   
 6   BonusMalus  678007 non-null  int32   
 7   VehBrand    678007 non-null  category
 8   VehGas      678007 non-null  category
 9   Density     678007 non-null  int32   
 10  Region      678007 non-null  category
 11  ClaimTotal  678007 non-null  float64 
 12  ClaimNb     678007 non-null  float64 
 13  LearnTest   678007 non-null  object  
dtypes: category(4), float64(4), int32(5), object(1)
memory usage: 41.4+ MB
None


### Preprocess Data for GLM

In [11]:
# Create GLM features
# Make Area code continuous
dat['AreaGLM'] = pd.Categorical(dat['Area']).codes+1
# Make vehicle power categorical
dat['VehPowerGLM'] = pd.Categorical(np.minimum(dat['VehPower'], 9))
# Create age categories
dat['VehAgeGLM'] = pd.cut(dat['VehAge'], 
                          bins=[0, 5, 12, 101],
                          labels=['0-5', '6-12', '12+'],
                          include_lowest=True)
# Create age categories
dat['DrivAgeGLM'] = pd.cut(dat['DrivAge'],
                           bins=[18, 20, 25, 30, 40, 50, 70, 101],
                           labels=['18-20', '21-25', '26-30', '31-40', '41-50', '51-70', '71+'],
                           include_lowest=True)
dat['DrivAgeGLM'] = pd.Categorical(dat['DrivAgeGLM'],
                                   categories=['31-40'] + [x for x in dat['DrivAgeGLM'].unique() if x != '31-40'])
# Censor bonus-malus level
dat['BonusMalusGLM'] = np.minimum(dat['BonusMalus'], 150)
# Consider density on log scale
dat['DensityGLM'] = np.log(dat['Density'])


# Keep other categorical variables as is
dat['VehBrand'] = pd.Categorical(dat['VehBrand'])
dat['VehGas'] = pd.Categorical(dat['VehGas'],
                              categories=['Diesel', 'Regular'])
dat['Region'] = pd.Categorical(dat['Region'])


### Split into Learning and Test Sets

In [12]:
# Split data (this uses the same split and order as in Wuthrich-Merz (Springer 2023))
learn = dat[dat['LearnTest'] == 'L'].copy()
test = dat[dat['LearnTest'] == 'T'].copy()

print(f"Learning set size: {len(learn)}")
print(f"Test set size: {len(test)}")

Learning set size: 610206
Test set size: 67801


### GLM Analysis

In [13]:
# Fit GLM
start_time = time.time()

model = glm("ClaimNb ~ DrivAgeGLM + VehBrand + VehGas + DensityGLM + AreaGLM",
            data=learn,
            offset=np.log(learn['Exposure']),
            family=sm.families.Poisson())

glm_results = model.fit()
print(f"Time taken: {time.time() - start_time:.2f} seconds\n")

# Display model summary
print(glm_results.summary())

Time taken: 2.18 seconds

                 Generalized Linear Model Regression Results                  
Dep. Variable:                ClaimNb   No. Observations:               610206
Model:                            GLM   Df Residuals:                   610186
Model Family:                 Poisson   Df Model:                           19
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -98513.
Date:                Mon, 03 Mar 2025   Deviance:                   1.5138e+05
Time:                        14:22:11   Pearson chi2:                 1.02e+06
No. Iterations:                     7   Pseudo R-squ. (CS):           0.004052
Covariance Type:            nonrobust                                         
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercep

### Calculate Deviance Losses

In [14]:
# Get predictions
learn['GLM'] = glm_results.predict(learn)*learn['Exposure']
test['GLM'] = glm_results.predict(test)*test['Exposure']

# Calculate in-sample and out-of-sample deviance
learn_deviance = poisson_deviance(learn['GLM'], learn['ClaimNb'])
test_deviance = poisson_deviance(test['GLM'], test['ClaimNb'])

print("Deviance Losses:")
print(f"Learning sample: {learn_deviance:.3f}")
print(f"Test sample: {test_deviance:.3f}")

Deviance Losses:
Learning sample: 24.807
Test sample: 25.027
